In [4]:
import import_ipynb
import chess
import torch
import torch.nn as nn
import torch.optim as optim
import dset
import net
import autoencoder
import bitboards

c_const = 2
mse = nn.MSELoss()

def cross_entropy(y_hat, y):
    y1, y2 = y[0], y[1]
    y_hat1 = (torch.clamp(y_hat[0], 1e-9, 1 - 1e-9))
    y_hat2 = (torch.clamp(y_hat[1], 1e-9, 1 - 1e-9))
    
    return -1/2 * ((y1 * torch.log(y_hat1)).sum(dim=1).mean() + (y2 * torch.log(y_hat2)).sum(dim=1).mean())

BATCH = 2
DATASET_SIZE = 20
ARGS = (chess.Board(), net.Net().cuda(), autoencoder.autoencoder().cuda(), dset.SearchType.MCTS, 50)
encoder = autoencoder.autoencoder().cuda()
nnet = net.Net().cuda()
optimizer = optim.Adam(nnet.parameters(), weight_decay=0.01)

dataset = dset.SearchDataset(DATASET_SIZE, dset.Encode(encoder), *ARGS)
DataLoader = torch.utils.data.DataLoader(dataset, batch_size=BATCH, shuffle=True)

for embedding, value, policy in DataLoader:
    value_hat, policy_hat = nnet(embedding.view(embedding.shape[0],1, 256))
    
    mse_value = mse(value_hat, value)
    cross_entropy_value = cross_entropy(policy_hat, policy)
    loss = c_const * mse_value + cross_entropy_value
    print(loss, mse_value, cross_entropy_value)
    
    loss.backward()
    optimizer.step()

tensor(0.1299, device='cuda:0', grad_fn=<AddBackward0>) tensor(3.7701e-05, device='cuda:0', grad_fn=<MseLossBackward>) tensor(0.1298, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.1297, device='cuda:0', grad_fn=<AddBackward0>) tensor(9.1268e-06, device='cuda:0', grad_fn=<MseLossBackward>) tensor(0.1297, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.1301, device='cuda:0', grad_fn=<AddBackward0>) tensor(6.9087e-09, device='cuda:0', grad_fn=<MseLossBackward>) tensor(0.1301, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.1296, device='cuda:0', grad_fn=<AddBackward0>) tensor(8.0523e-06, device='cuda:0', grad_fn=<MseLossBackward>) tensor(0.1296, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.1300, device='cuda:0', grad_fn=<AddBackward0>) tensor(2.5393e-05, device='cuda:0', grad_fn=<MseLossBackward>) tensor(0.1299, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.1302, device='cuda:0', grad_fn=<AddBackward0>) tensor(3.4305e-05, device='cuda:0', grad_fn=<MseLossBackward>) tensor